In [7]:
pip install munch

In [8]:
from munch import DefaultMunch
import os
import glob
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.utils import save_image

# Configurations
Config = {
    'data_path' : '/content/Dataset',
    'year': "2022",
    'img_size' : 128,
    'batch_size_GAN' : 64,
    'epochs_GAN': 10000,
    'lr_GAN': 0.0001,
    'latent_dim' : 32,
    'n_critic': 5,
    'clip_value':0.01,
    'load_G_name' : 'my_G3900.pth',
    'epochs_lstm': 30000,
    'batch_size_lstm': 4,
    'lr_lstm': 0.01,
    'J': 8,
    'hidden_size' : 64,
    'layer': 2,
}
opt = DefaultMunch.fromDict(Config)

# Model Definitions
img_shape = (1, opt.img_size, opt.img_size)

class Generator(nn.Module):
    def __init__(self, input_dim):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(input_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            *block(1024, 2048),
            *block(2048, 4096),
            nn.Linear(4096, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 2048),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(2048, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

class Dataset_m(Dataset):
    def __init__(self, root, transform):
        self.root = root
        self.transform = transform
        self.data_list = glob.glob(f'{self.root}/*jpg')
        print(f"Found {len(self.data_list)} images in {self.root}")

    def __getitem__(self, index):
        data = self.data_list[index]
        data = Image.open(data).convert('L')
        data = self.transform(data)
        return data

    def __len__(self):
        return len(self.data_list)

# ---------- Main ----------
if __name__ == '__main__':
    os.makedirs("images", exist_ok=True)
    print(opt)

    cuda = True if torch.cuda.is_available() else False

    generator = Generator(input_dim=opt.latent_dim).cuda()
    discriminator = Discriminator().cuda()

    transforms_train = transforms.Compose([
        transforms.Resize((opt.img_size, opt.img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=0.5, std=0.5),
    ])

    my_dataset = Dataset_m(f'{opt.data_path}', transform=transforms_train)
    dataloader = DataLoader(dataset=my_dataset, batch_size=opt.batch_size_GAN, shuffle=True)

    optimizer_G = torch.optim.RMSprop(generator.parameters(), lr=opt.lr_GAN)
    optimizer_D = torch.optim.RMSprop(discriminator.parameters(), lr=opt.lr_GAN)

    final_gen_images = []  # To store final 50 images

    for epoch in range(opt.epochs_GAN):
        for i, imgs in enumerate(dataloader):

            real_imgs = imgs.cuda()
            iteration = epoch * len(dataloader) + i

            # Train Discriminator
            optimizer_D.zero_grad()
            z = torch.randn(imgs.shape[0], opt.latent_dim).cuda()
            fake_imgs = generator(z).detach()
            fake_of_D = torch.mean(discriminator(fake_imgs))
            true_of_D = torch.mean(discriminator(real_imgs))
            loss_D = fake_of_D - true_of_D
            loss_D.backward()
            optimizer_D.step()

            for p in discriminator.parameters():
                p.data.clamp_(-opt.clip_value, opt.clip_value)

            # Train Generator every n_critic steps
            if i % opt.n_critic == 0:
                optimizer_G.zero_grad()
                gen_imgs = generator(z)
                loss_G = -torch.mean(discriminator(gen_imgs))
                loss_G.backward()
                optimizer_G.step()

                if iteration % 100 == 0:
                    print(f"[Epoch {epoch}/{opt.epochs_GAN}] [Iter {iteration}] [D loss: {loss_D.item()}] [G loss: {loss_G.item()}]")

        # Save model and generated images every 150 epochs
        if (epoch + 1) % 150 == 0:
            save_model_path = './saved_model'
            os.makedirs(save_model_path, exist_ok=True)
            torch.save(generator.state_dict(), os.path.join(save_model_path, f'my_G{epoch+1}.pth'))

            for j, img in enumerate(gen_imgs[:len(my_dataset)]):  # Save separate images
                save_image(img, f"images/epoch_{epoch+1}_img_{j+1}.png", normalize=True)

        # Save final images (last 50 epochs)
        if epoch >= opt.epochs_GAN - 50:
            final_gen_images.append(gen_imgs[0].detach().cpu())  # Save only one per epoch

    # After training: Save final 50 images
    for idx, img in enumerate(final_gen_images):
        save_image(img, f"images/final_{idx+1}.png", normalize=True)


DefaultMunch(None, {'data_path': '/content/Dataset', 'year': '2022', 'img_size': 128, 'batch_size_GAN': 64, 'epochs_GAN': 10000, 'lr_GAN': 0.0001, 'latent_dim': 32, 'n_critic': 5, 'clip_value': 0.01, 'load_G_name': 'my_G3900.pth', 'epochs_lstm': 30000, 'batch_size_lstm': 4, 'lr_lstm': 0.01, 'J': 8, 'hidden_size': 64, 'layer': 2})
Found 6 images in /content/Dataset
[Epoch 0/10000] [Iter 0] [D loss: 0.005471652373671532] [G loss: 0.008700779639184475]
[Epoch 100/10000] [Iter 100] [D loss: -71.4735107421875] [G loss: -168.00198364257812]
[Epoch 200/10000] [Iter 200] [D loss: -108.00687408447266] [G loss: 95.29097747802734]
[Epoch 300/10000] [Iter 300] [D loss: -73.1285400390625] [G loss: 671.4341430664062]
[Epoch 400/10000] [Iter 400] [D loss: -189.16217041015625] [G loss: 111.56846618652344]
[Epoch 500/10000] [Iter 500] [D loss: -105.32147216796875] [G loss: -361.9626159667969]
[Epoch 600/10000] [Iter 600] [D loss: -98.5716552734375] [G loss: -423.93853759765625]
[Epoch 700/10000] [Iter 

In [9]:
import glob
import zipfile
import os

# Step 1: Find only images that start with "final"
final_images = glob.glob('/content/images/final*.png')
print(f"Total FINAL images: {len(final_images)}")
print("First 5 FINAL images:", final_images[:5])

# Step 2: Create a zip file with only final images
zip_path = "/content/final_images.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in final_images:
        zipf.write(file, arcname=os.path.basename(file))  # Store only the filename, not full path

# Step 3: Download the zip file
from google.colab import files
files.download(zip_path)


Total FINAL images: 50
First 5 FINAL images: ['/content/images/final_3.png', '/content/images/final_46.png', '/content/images/final_26.png', '/content/images/final_33.png', '/content/images/final_22.png']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>